## Optimize the Model


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 

charity1_df = pd.read_csv("Resources/charity_data.csv")
charity1_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [27]:
# Drop the non-beneficial EIN column

charity2_df = charity1_df.drop(['EIN'], axis=1, inplace=False)
charity2_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [28]:
# display the number of unique values

for col in charity2_df.columns:
    print(col, len(charity2_df[col].unique().tolist()))

NAME 19568
APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [29]:
# Look at NAME value counts for binning

name_types = charity2_df['NAME'].value_counts()
name_types

PARENT BOOSTER USA INC                                          1260
TOPS CLUB INC                                                    765
UNITED STATES BOWLING CONGRESS INC                               700
WASHINGTON STATE UNIVERSITY                                      492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                  408
                                                                ... 
RONALD MCDONALD HOUSE CHARITIES INC                                1
WHARTON COUNTY STRAY PET OUTREACH TEAM                             1
INTERNATIONAL COMMUNITY SCHOOLS INC                                1
INTERNATIONAL HOSPITALITY INFORMATION TECHNOLOGY ASSOCIATION       1
CHRISTUS SCHUMPERT MEDICAL CENTER                                  1
Name: NAME, Length: 19568, dtype: int64

In [26]:
name_types[name_types > 45]

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [31]:
# Determine which values to replace if counts are less than or equal to 5.

name_types_to_replace = list(name_types[name_types <= 5].index)

# Replace in dataframe
for name in name_types_to_replace:
    charity2_df['NAME'] = charity2_df['NAME'].replace(name,"Other")
    
# Check to make sure binning was successful
charity2_df['NAME'].value_counts()

Other                                                         20043
PARENT BOOSTER USA INC                                         1260
TOPS CLUB INC                                                   765
UNITED STATES BOWLING CONGRESS INC                              700
WASHINGTON STATE UNIVERSITY                                     492
                                                              ...  
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC                         6
DELTA KAPPA GAMMA SOCIETY INTERNATIONAL                           6
AMERICAN INDEPENDENT BUSINESS ALLIANCE                            6
OFFICE AND PROFESSIONAL EMPLOYEES INTERNATIONAL UNION             6
DEPARTMENT OF OREGON LADIES AUXILLARY TO THE VFW OF THE US        6
Name: NAME, Length: 355, dtype: int64

In [32]:
# Look at APPLICATION_TYPE value counts for binning

application_types = charity2_df['APPLICATION_TYPE'].value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Determine which values to replace if application counts are less than 500

application_types_to_replace = list(application_types[application_types < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    charity2_df['APPLICATION_TYPE'] = charity2_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
charity2_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [34]:
# Look at CLASSIFICATION value counts for binning

class_types = charity2_df['CLASSIFICATION'].value_counts()
class_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1236        1
C6100        1
C1370        1
C4120        1
C1728        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [35]:
# Determine which values to replace if counts are less than 1000

class_types_to_replace = list(class_types[class_types < 1000].index)

# Replace in dataframe
for cls in class_types_to_replace:
    charity2_df['CLASSIFICATION'] = charity2_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
charity2_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [41]:
# Generate our categorical variable lists

charity2_df.columns.tolist()

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'STATUS',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS',
 'ASK_AMT',
 'IS_SUCCESSFUL']

In [42]:
# perform one-hot encoding on the data

charity3_df = pd.get_dummies(charity2_df)
charity3_df.shape

(34299, 399)

In [43]:
# Split our preprocessed data into our features and target arrays

y = charity3_df['IS_SUCCESSFUL'].values
X = charity3_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [44]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

len(X_train[0])

398

In [46]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               39900     
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 10)                310       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 43,251
Trainable params: 43,251
Non-trainable params: 0
_________________________________________________________________


In [47]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [48]:
# Train the model

fit_model = nn.fit(X_train_scaled,y_train,epochs=100, verbose=1)

Epoch 1/100
804/804 [==============================] - 1s 724us/step - loss: 0.4967 - accuracy: 0.7715
Epoch 2/100
804/804 [==============================] - 1s 713us/step - loss: 0.4402 - accuracy: 0.7946
Epoch 3/100
804/804 [==============================] - 1s 736us/step - loss: 0.4322 - accuracy: 0.7965
Epoch 4/100
804/804 [==============================] - 1s 727us/step - loss: 0.4282 - accuracy: 0.7999
Epoch 5/100
804/804 [==============================] - 1s 740us/step - loss: 0.4251 - accuracy: 0.7986
Epoch 6/100
804/804 [==============================] - 1s 708us/step - loss: 0.4234 - accuracy: 0.7983
Epoch 7/100
804/804 [==============================] - 1s 706us/step - loss: 0.4221 - accuracy: 0.7989
Epoch 8/100
804/804 [==============================] - 1s 706us/step - loss: 0.4209 - accuracy: 0.8003
Epoch 9/100
804/804 [==============================] - 1s 711us/step - loss: 0.4196 - accuracy: 0.8007
Epoch 10/100
804/804 [==============================] - 1s 708us/step - l

804/804 [==============================] - 1s 762us/step - loss: 0.3996 - accuracy: 0.8084
Epoch 80/100
804/804 [==============================] - 1s 755us/step - loss: 0.3999 - accuracy: 0.8085
Epoch 81/100
804/804 [==============================] - 1s 757us/step - loss: 0.4004 - accuracy: 0.8088
Epoch 82/100
804/804 [==============================] - 1s 767us/step - loss: 0.4000 - accuracy: 0.8085
Epoch 83/100
804/804 [==============================] - 1s 764us/step - loss: 0.4000 - accuracy: 0.8092
Epoch 84/100
804/804 [==============================] - 1s 787us/step - loss: 0.3995 - accuracy: 0.8084
Epoch 85/100
804/804 [==============================] - 1s 786us/step - loss: 0.3997 - accuracy: 0.8078
Epoch 86/100
804/804 [==============================] - 1s 767us/step - loss: 0.3993 - accuracy: 0.8080
Epoch 87/100
804/804 [==============================] - 1s 775us/step - loss: 0.3999 - accuracy: 0.8085
Epoch 88/100
804/804 [==============================] - 1s 766us/step - loss:

In [57]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss*100:2.1f}%, Accuracy: {model_accuracy*100:2.1f}%")

268/268 - 0s - loss: 0.4714 - accuracy: 0.7866 - 131ms/epoch - 487us/step
Loss: 47.1%, Accuracy: 78.7%


In [58]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

USE A RANDOM FOREST CLASSIFIER ON THE DATASET

In [60]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [61]:
# Create a random forest classifier.

rf_model = RandomForestClassifier(n_estimators=128, random_state=42)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred)*100:2.1f}%")

Accuracy: 77.2%
